In [ ]:
# !pip install numpy --upgrade
# !pip install mahotas

import os
import cv2
import glob
import random
import mahotas
import numpy as np

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, LabelEncoder

from skimage.exposure import is_low_contrast
from skimage.filters import gabor_kernel
from scipy.stats import kurtosis, skew
from matplotlib import pyplot as plt
from skimage import exposure, feature, color
from skimage.feature import hog
from scipy import ndimage as ndi
from pywt import dwt2
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier


from load_data_metrics import load_data, metrics, cv_train, results, data_balanced


# Features extract

In [ ]:
kernels = []
for theta in range(2):
    theta = theta / 4. * np.pi
    for sigma in (2,3):
        for frequency in (0.05, 0.25):
            kernel = np.real(gabor_kernel(frequency, theta=theta,
                                              sigma_x=sigma, sigma_y=sigma))
            kernels.append(kernel)

def gabor_feat(image):

    feats = np.zeros((len(kernels), 3), dtype=np.double)
    for k, kernel in enumerate(kernels):
        filtered = ndi.convolve(image, kernel, mode='wrap')
        feats[k, 0] = filtered.mean() # Média
        feats[k, 1] = skew(filtered.flatten()) # Assimetria
        feats[k, 2] = kurtosis(filtered.flatten()) # Curtose
        _, (cH, cV, cD) = dwt2(filtered.T, 'db1')
        r_feat = np.hstack([feats[0],feats[1],feats[2],feats[3]])

    return r_feat 



# Momentos Zernike 
def ZernikeMoments(image):
    gray = image
    rows, cols = gray.shape
    radius = cols//2 if rows > cols else rows//2
    zernike = mahotas.features.zernike_moments(gray,radius)
    return zernike    


# Histogram of Oriented Gradients (HOG)
def fd_hog_gradients(image):

    fd, hog_image = feature.hog(image, orientations=8, pixels_per_cell=(128, 128),cells_per_block=(2, 2),
                                transform_sqrt=True , block_norm="L1",visualize=True)
    return fd


# LBP - Local Binary Pattern 
radius = 8
numPoints = 64
def describeLBP(image, eps=1e-7):
    lbp = feature.local_binary_pattern(image,numPoints,radius, method="uniform")
    (hist, _) = np.histogram(lbp.ravel(),bins=np.arange(0, numPoints + 3),range=(0, numPoints + 2))
    return hist


# Load dataset


In [ ]:
def load_path(path):
    labels_ = os.listdir(path)
    features = []
    labels   = []
    for i, label in enumerate(labels_):
        cur_path = path + "/" + label 
        for image_path in glob.glob(cur_path + "/*"):
            img = cv2.imread(image_path)
            image = cv2.resize(img, (256,256))
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            
            image2 = image/255
            lbp = describeLBP(image2) # LBP
            hog = fd_hog_gradients(image2) # HOG 
            zernike = ZernikeMoments(image2) # Momentos de Zernike
            gabor= gabor_feat(image2) # Filtros de Gabor 

            global_feature = np.hstack([hog,lbp, zernike, gabor])
            features.append(global_feature)
            labels.append(label)            
        print ("[INFO] completed label - " + label)
    return features, labels


data, labels = load_data(load_path)

# Classify

In [ ]:
svm = SVC(C=3, kernel='rbf', probability=True)
mlp = MLPClassifier(activation='relu',max_iter=500,alpha=0.001,hidden_layer_sizes=(40,),learning_rate='constant')
xgb = XGBClassifier(colsample_bytree= 1, max_depth=5, n_estimators=300, subsample= 0.6, seed=0)

models = []
models.append(('SVM', SVC(C=3, kernel='rbf')))
models.append(('XGB', XGBClassifier(colsample_bytree= 1, max_depth=5, n_estimators= 100, subsample= 0.6, seed=0))) 
models.append(('MLP', MLPClassifier(activation='relu',max_iter=1000,alpha=0.01,hidden_layer_sizes=(40,40,40),learning_rate='constant')))
models.append(('VOT', VotingClassifier(estimators = [('svm', svm),('mlp', mlp),('xgb', xgb)], voting='hard')))

# Results

## Unbalanced

In [ ]:
P0, A0, S0, E0, Fs0, TP0, TN0, FP0, FN0 = cv_train(models, data, labels, 10, 'None', 'scaler')
results(P0, A0, S0, E0, Fs0, TP0, TN0, FP0, FN0)

## Random Balanced

In [ ]:
P3, A3, S3, E3, Fs3, TP3, TN3, FP3, FN3 = cv_train(models, data, labels, 10, 'random', 'scaler')
results(P3, A3, S3, E3, Fs3, TP3, TN3, FP3, FN3)


## Near Miss Balanced

In [ ]:
P2, A2, S2, E2, Fs2, TP2, TN2, FP2, FN2 = cv_train(models, data, labels, 10, 'nm', 'scaler')
results(P2, A2, S2, E2, Fs2, TP2, TN2, FP2, FN2)


## Cluster Centroid Balanced

In [ ]:
P1, A1, S1, E1, Fs1, TP1, TN1, FP1, FN1 = cv_train(models, data, labels, 10, 'cluster', 'scaler')
results(P1, A1, S1, E1, Fs1, TP1, TN1, FP1, FN1)
